---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas
import numpy

In [2]:
spam_data = pandas.read_csv('spam.csv')

spam_data['target'] = numpy.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

                                                text  target
0  Go until jurong point, crazy.. Available only ...       0
1                      Ok lar... Joking wif u oni...       0
2  Free entry in 2 a wkly comp to win FA Cup fina...       1
3  U dun say so early hor... U c already then say...       0
4  Nah I don't think he goes to usf, he lives aro...       0
5  FreeMsg Hey there darling it's been 3 week's n...       1
6  Even my brother is not like to speak with me. ...       0
7  As per your request 'Melle Melle (Oru Minnamin...       0
8  WINNER!! As a valued network customer you have...       1
9  Had your mobile 11 months or more? U R entitle...       1

In [3]:
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [4]:
def answer_one():
    return 100 * sum(spam_data.target)/float(len(spam_data))

In [5]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(X_train)
X_test_count = vectorizer.transform(X_test)

def answer_two():
    return max(((len(token), token) for token in vectorizer.vocabulary_))[1]    

In [7]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_count, y_train)
    predictions = model.predict(X_test_count)
    return roc_auc_score(y_test, predictions)

In [9]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_transform = TfidfVectorizer()
X_train_tfidf = tfidf_transform.fit_transform(X_train)
X_train_tfidf_frame = pandas.DataFrame(
    X_train_tfidf.toarray(),
    columns=tfidf_transform.get_feature_names())

def answer_four():
    tfidfs = X_train_tfidf_frame.sum()
    tfidfs_ascending = tfidfs.sort_values()
    return (tfidfs_ascending.head(n=20),
            tfidfs_ascending.tail(n=20).sort_values(ascending=False))

In [11]:
answer_four()

(aaniye          0.074475
 exterminator    0.074475
 venaam          0.074475
 organizer       0.074475
 courageous      0.074475
 dependable      0.074475
 stylist         0.074475
 psychiatrist    0.074475
 determined      0.074475
 pest            0.074475
 psychologist    0.074475
 sympathetic     0.074475
 listener        0.074475
 healer          0.074475
 pudunga         0.074475
 chef            0.074475
 athletic        0.074475
 companion       0.074475
 childrens       0.091250
 shivratri       0.091250
 dtype: float64, you     185.842841
 to      154.596529
 the     109.931059
 in       95.998208
 me       87.272670
 and      83.902976
 is       83.665139
 my       78.240656
 it       78.235079
 call     74.859833
 ok       70.873379
 for      69.774971
 your     69.088919
 that     67.316022
 have     66.216315
 are      64.431131
 can      61.403843
 now      61.174895
 of       60.723502
 on       57.845792
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
tfidf_3 = TfidfVectorizer(min_df=3)
X_train_tfidf_3 = tfidf_3.fit_transform(X_train)
X_test_tfidf_3 = tfidf_3.transform(X_test)

def answer_five():
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_tfidf_3, y_train)
    predictions = model.predict(X_test_tfidf_3)
    return roc_auc_score(y_test, predictions)

In [13]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [14]:
SPAM = spam_data[spam_data.target == 1]
NOT_SPAM = spam_data[spam_data.target == 0]

def answer_six():
    return (numpy.mean(NOT_SPAM.text.str.len()), numpy.mean(SPAM.text.str.len()))

In [15]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [16]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
from sklearn.svm import SVC

def answer_seven():
    tfidf = TfidfVectorizer(min_df=5)
    x_train_tfidf = tfidf.fit_transform(X_train)
    x_test_tfidf = tfidf.transform(X_test)
    model = SVC(C=10000)
    model.fit(x_train_tfidf, y_train)
    predictions = model.predict(x_test_tfidf)
    return roc_auc_score(y_test, predictions)

In [18]:
answer_seven()

0.94971605860482489

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [19]:
def answer_eight():
    return (numpy.mean(NOT_SPAM.text.str.count("(\d)")),
            numpy.mean(SPAM.text.str.count("(\d)")))

In [20]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [21]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
    x_train_tfidf = tfidf.fit_transform(X_train)
    x_test_tfidf = tfidf.transform(X_test)

    x_reset = X_train.reset_index(drop=True)

    x_train = pandas.DataFrame(x_train_tfidf.toarray())
    x_train["length"] = x_reset.str.len()
    x_train["digits"] = x_reset.str.count("(\d)")
    
    model = LogisticRegression(C=100)
    model.fit(x_train, y_train)

    x_test_reset = X_test.reset_index(drop=True)
    x_test = pandas.DataFrame(x_test_tfidf.toarray())
    x_test["length"] = x_test_reset.str.len()
    x_test["digits"] = x_test_reset.str.count("(\d)")

    predictions= model.predict(x_test)
    return roc_auc_score(y_test, predictions)

In [22]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [23]:
def answer_ten():
    return (numpy.mean(NOT_SPAM.text.str.count("\W")),
            numpy.mean(SPAM.text.str.count("(\W)")))

In [24]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [31]:
def answer_eleven():
    """Fits a Logistic Regression Model using the CountVectorizer

    CountVectorizer:
     ,* ignore document frequency lower than 5
     ,* n-grams from 2 to 5
     ,* analyzer ``char_wb`` (character n-grams only within word boundaries)

    Added features:
     ,* length_of_doc (number of characters in the document)
     ,* digit_count (number of digits in the document)
     ,* non_word_char_count (number for non-alphanumeric characters)

    Returns:
     tuple: (AUC score, names of 10 smallest coefficients, 
             names of 10 largest coefficients)
    """
    vectorizer = CountVectorizer(min_df=5, ngram_range=(2, 5),
                                 analyzer="char_wb")

    x_train_ = vectorizer.fit_transform(X_train)
    x_test_ = vectorizer.transform(X_test)
    x_train = pandas.DataFrame(x_train_.toarray(),
                               columns=vectorizer.get_feature_names())

    x_reset = X_train.reset_index(drop=True)

    x_train["length_of_doc"] = x_reset.str.len()
    x_train["digit_count"] = x_reset.str.count("(\d)")
    x_train["non_word_char_count"] = x_reset.str.count("(\W)")

    model = LogisticRegression(C=100)
    model.fit(x_train, y_train)

    x_test_reset = X_test.reset_index(drop=True)

    x_test = pandas.DataFrame(x_test_.toarray(), columns=vectorizer.get_feature_names())
    x_test["length_of_doc"] = x_test_reset.str.len()
    x_test["digit_count"] = x_test_reset.str.count("(\d)")
    x_test["non_word_char_count"] = x_test_reset.str.count("(\W)")

    predictions = model.predict(x_test)
    score = roc_auc_score(y_test, predictions)

    coefficients = pandas.DataFrame(dict(coefficient=model.coef_[0],
                                         name=x_train.columns))
    smallest = coefficients.sort_values(by='coefficient').head(n=10)
    largest = coefficients.sort_values(by='coefficient',
                                       ascending=False).head(n=10)
    return score, list(smallest.name), list(largest.name)

In [32]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])